In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_character = pd.read_csv(os.getenv('SIMPSONS_CHARACTERS'))
df_lines = pd.read_csv(os.getenv('SIMPSONS_SCRIPT_LINES'), low_memory=False)
df_episode = pd.read_csv(os.getenv('SIMPSONS_EPISODES'))
df_location = pd.read_csv(os.getenv('SIMPSONS_LOCATIONS'))

### Pre-Processing

In [2]:
# It is important to check data type for each column and do neccessary conversion.
# (e.g. speaking_line is object not boolean)
# (e.g. Non-Null Count can show which columns have null values)
df_lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158271 entries, 0 to 158270
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  158271 non-null  int64  
 1   episode_id          158271 non-null  int64  
 2   number              158271 non-null  int64  
 3   raw_text            158271 non-null  object 
 4   timestamp_in_ms     158271 non-null  object 
 5   speaking_line       158271 non-null  object 
 6   character_id        140750 non-null  object 
 7   location_id         157864 non-null  float64
 8   raw_character_text  140749 non-null  object 
 9   raw_location_text   157863 non-null  object 
 10  spoken_words        132112 non-null  object 
 11  normalized_text     132087 non-null  object 
 12  word_count          132112 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 15.7+ MB


In [3]:
df_lines[df_lines.speaking_line=='false']

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
8,9557,32,217,(Apartment Building: Ext. apartment building -...,889000,false,NaN,374.0,NaN,Apartment Building,NaN,NaN,NaN
16,9565,32,225,(Springfield Elementary School: EXT. ELEMENTAR...,918000,false,NaN,3.0,NaN,Springfield Elementary School,NaN,NaN,NaN
27,9576,32,236,Bart Simpson: (ANGUISHED SCREAM),936000,false,8,3.0,Bart Simpson,Springfield Elementary School,NaN,NaN,NaN
29,75766,263,106,(Moe's Tavern: Int. Moe's - evening),497000,false,NaN,15.0,NaN,Moe's Tavern,NaN,NaN,NaN
35,9583,32,243,(Train Station: int. train station - afternoon),960000,false,NaN,375.0,NaN,Train Station,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
158194,9472,32,132,(Springfield Elementary School: INT. ELEMENTAR...,601000,false,NaN,3.0,NaN,Springfield Elementary School,NaN,NaN,NaN
158205,9483,32,143,(Simpson Home: INT. SIMPSON HOUSE - LIVINGING ...,646000,false,NaN,5.0,NaN,Simpson Home,NaN,NaN,NaN
158215,9493,32,153,(Museum: INT. MUSEUM - ADMISSION AREA - DAY),686000,false,NaN,372.0,NaN,Museum,NaN,NaN,NaN
158243,9521,32,181,(SIMPSON HOUSE - BASEMENT STAIRCASE: INT. SIMP...,789000,false,NaN,373.0,NaN,SIMPSON HOUSE - BASEMENT STAIRCASE,NaN,NaN,NaN


In [4]:
df_lines[df_lines.speaking_line=='false'].iloc[1]['raw_text']

'(Springfield Elementary School: EXT. ELEMENTARY - SCHOOL PLAYGROUND - AFTERNOON)'

In [5]:
df_lines = df_lines.dropna(subset=['character_id'])
df_lines.shape

(140750, 13)

In [6]:
df_lines.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140750 entries, 0 to 158270
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  140750 non-null  int64  
 1   episode_id          140750 non-null  int64  
 2   number              140750 non-null  int64  
 3   raw_text            140750 non-null  object 
 4   timestamp_in_ms     140750 non-null  object 
 5   speaking_line       140750 non-null  object 
 6   character_id        140750 non-null  object 
 7   location_id         140343 non-null  float64
 8   raw_character_text  140749 non-null  object 
 9   raw_location_text   140342 non-null  object 
 10  spoken_words        132110 non-null  object 
 11  normalized_text     132085 non-null  object 
 12  word_count          132110 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 15.0+ MB


In [7]:
# fill NaN spoken words column with empty string 
df_lines['spoken_words'] = df_lines['spoken_words'].fillna('')
# count character lines overall (you can either groupby character_id or raw_character_text):
character_lines_count_by_episode = df_lines.groupby(['character_id', 'episode_id']).count()[['spoken_words']].reset_index()
character_lines_count_by_episode.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21288 entries, 0 to 21287
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   character_id  21288 non-null  object
 1   episode_id    21288 non-null  int64 
 2   spoken_words  21288 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 499.1+ KB


In [8]:
character_lines_count_by_episode.head()

,character_id,episode_id,spoken_words
0,1,1,43
1,1,2,23
2,1,3,15
3,1,4,41
4,1,5,17


In [9]:
df_character.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6722 entries, 0 to 6721
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               6722 non-null   int64 
 1   name             6722 non-null   object
 2   normalized_name  6722 non-null   object
 3   gender           323 non-null    object
dtypes: int64(1), object(3)
memory usage: 210.2+ KB


In [10]:
def convert_to_int(x):
    try:
        return int(x)
    except:
        return -999

character_lines_count_by_episode['character_id'] = character_lines_count_by_episode['character_id'].apply(convert_to_int)
# merge 2 datasets and clean up column names
character_lines_count_by_episode = character_lines_count_by_episode.merge(df_character, left_on='character_id', right_on='id', how='left')
character_lines_count_by_episode = character_lines_count_by_episode.drop('id', axis=1).sort_values('character_id')
character_lines_count_by_episode = character_lines_count_by_episode.rename(columns={'spoken_words':'lines_count'})

In [11]:
# sort characters by number of lines they have spoken
character_lines_count_by_episode = character_lines_count_by_episode.sort_values('lines_count', ascending=False)

In [12]:
# Not surprising that Homer Simpsons have the most lines in most cases
character_lines_count_by_episode.head()

,character_id,episode_id,lines_count,name,normalized_name,gender
6752,2,231,145,Homer Simpson,homer simpson,m
6545,2,24,129,Homer Simpson,homer simpson,m
19145,8,34,122,Bart Simpson,bart simpson,m
6872,2,351,122,Homer Simpson,homer simpson,m
6814,2,293,119,Homer Simpson,homer simpson,m


In [13]:
character_lines_count_by_episode[character_lines_count_by_episode.gender.isnull()]

,character_id,episode_id,lines_count,name,normalized_name,gender
16531,5669,465,49,Kurt Hardwick,kurt hardwick,NaN
17041,5996,498,40,Bar Rag,bar rag,NaN
13842,4174,333,30,TEN-YEAR-OLD MARGE,ten-year-old marge,NaN
18024,6666,559,29,MR. LASSEN,mr lassen,NaN
8136,2251,413,28,Donny,donny,NaN
...,...,...,...,...,...,...
17904,658,214,1,Everyone,everyone,NaN
17906,658,283,1,Everyone,everyone,NaN
17907,658,294,1,Everyone,everyone,NaN
17909,658,470,1,Everyone,everyone,NaN


In [14]:
character_lines_count_by_episode = character_lines_count_by_episode.pivot(index=['name'], columns = ['episode_id'])[['lines_count']]
character_lines_count_by_episode = character_lines_count_by_episode['lines_count'].fillna(0.0)

In [15]:
character_lines_count_by_episode

episode_id,1,2,3,4,5,6,7,8,9,10,...,559,560,561,562,563,564,565,566,567,568
name,,,,,,,,,,,,,,,,,,,,,
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""For Dummies"" Author",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Just Stamp the Ticket"" Man",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Mario"" #2",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Shorts"" Bart",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
alligator,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iPod,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
newsstand Operator,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
top_n = 300
top_characters = list(character_lines_count_by_episode.sum(axis=1).sort_values(ascending=False)[:top_n].index)
top_characters

['Homer Simpson',
 'Marge Simpson',
 'Bart Simpson',
 'Lisa Simpson',
 'C. Montgomery Burns',
 'Moe Szyslak',
 'Seymour Skinner',
 'Ned Flanders',
 'Grampa Simpson',
 'Milhouse Van Houten',
 'Chief Wiggum',
 'Krusty the Clown',
 'Nelson Muntz',
 'Lenny Leonard',
 'Apu Nahasapeemapetilon',
 'Waylon Smithers',
 'Kent Brockman',
 'Carl Carlson',
 'Edna Krabappel-Flanders',
 'Dr. Julius Hibbert',
 'Barney Gumble',
 'Selma Bouvier',
 'Sideshow Bob',
 'Rev. Timothy Lovejoy',
 'Groundskeeper Willie',
 'Crowd',
 'Gary Chalmers',
 'Ralph Wiggum',
 'Mayor Joe Quimby',
 'Comic Book Guy',
 'Patty Bouvier',
 'Otto Mann',
 'Martin Prince',
 'Announcer',
 'Kids',
 'Jimbo Jones',
 'Sideshow Mel',
 'Lou',
 'Professor Jonathan Frink',
 'Fat Tony',
 'Kearney Zzyzwicz',
 'Agnes Skinner',
 'Snake Jailbird',
 'Kirk Van Houten',
 'Cletus Spuckler',
 'Troy McClure',
 'DOLPH',
 'Todd Flanders',
 'Audience',
 'Lionel Hutz',
 'Miss Hoover',
 'Gil Gunderson',
 'Rainier Wolfcastle',
 'Narrator',
 'The Rich Texan',

In [17]:
character_lines_count_by_episode_filtered = character_lines_count_by_episode[character_lines_count_by_episode.index.isin(top_characters)]

In [18]:
character_lines_count_by_episode_filtered

episode_id,1,2,3,4,5,6,7,8,9,10,...,559,560,561,562,563,564,565,566,567,568
name,,,,,,,,,,,,,,,,,,,,,
10-Year-Old Homer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARTIE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Agnes Skinner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
Akira,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alec Baldwin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wayne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Woman,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Women,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_episode.shape

(600, 14)

In [20]:
len(df_lines.episode_id.unique())

564